# Notebook for this project

## Training the model

In [2]:
import logging
from prefect import task, flow
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import mlflow
import numpy as np

logging.basicConfig(level=logging.INFO)

In [3]:
def prepare_data(data):
    X = data.drop(["DEATH_EVENT"], axis=1)
    y = data["DEATH_EVENT"]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

def initialise_model(input_dim: int) -> Sequential:
    model = Sequential([
        Dense(16, activation='relu', input_dim=input_dim),
        Dense(8, activation='relu'),
        Dropout(0.25),
        Dense(4, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train, model):
    early_stopping = EarlyStopping(min_delta=0.001, patience=20, restore_best_weights=True)
    history = model.fit(X_train, y_train, batch_size=32, epochs=500, callbacks=[early_stopping], validation_split=0.2)
    return history

def train_model_flow(data):
    with mlflow.start_run():
        X_train, y_train = prepare_data(data)
        model = initialise_model(X_train.shape[1])
        history = train_model(X_train, y_train, model)
        logging.info(f"Validation Accuracy: {np.mean(history.history['val_accuracy']) * 100:.2f}%")
    return model

train_model_flow()
